# Find feature = furnished

In [23]:
"""
A very simple and basic web scraping script. Feel free to
use this as a source of inspiration, but, make sure to attribute
it if you do so.
This is by no means production code.
"""
# built-in imports
import re
from json import dump

from collections import defaultdict

# user packages
from bs4 import BeautifulSoup, SoupStrainer
from urllib.request import urlopen
import requests

# constants
BASE_URL = "https://www.domain.com.au"
N_PAGES = range(1, 2) # update this to your liking

# begin code
url_links = []
property_metadata = defaultdict(dict)
headers = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}
# generate list of urls to visit
for page in N_PAGES:
    url = BASE_URL + f"/rent/melbourne-region-vic/?features=furnished&sort=price-desc&page={page}"
    bs_object = BeautifulSoup(requests.get(url,headers = headers).text, "html.parser")
    # find the unordered list (ul) elements which are the results, then
    # find all href (a) tags that are from the base_url website.
    if bs_object is not None:
        index_links = bs_object \
            .find(
                "ul",
                {"data-testid": "results"}
            ) \
            .findAll(
                "a",
                href=re.compile(f"{BASE_URL}/*") # the `*` denotes wildcard any
            )

    for link in index_links:
        # if its a property address, add it to the list
        if 'address' in link['class']:
            url_links.append(link['href'])

In [24]:
# for each url, scrape some basic metadata
for property_url in url_links[1:]:
    bs_object = BeautifulSoup(requests.get(property_url,headers = headers).text, "html.parser")

    # looks for the header class to get property name
    property_metadata[property_url]['name'] = bs_object \
        .find("h1", {"class": "css-164r41r"}) \
        .text

    # looks for the div containing a summary title for cost
    property_metadata[property_url]['cost_text'] = bs_object \
        .find("div", {"data-testid": "listing-details__summary-title"}) \
        .text

    # extract coordinates from the hyperlink provided
    # i'll let you figure out what this does :P
    property_metadata[property_url]['coordinates'] = [
        float(coord) for coord in re.findall(
            r'destination=([-\s,\d\.]+)', # use regex101.com here if you need to
            bs_object \
                .find(
                    "a",
                    {"target": "_blank", 'rel': "noopener noreferer"}
                ) \
                .attrs['href']
        )[0].split(',')
    ]
    '''
    property_metadata[property_url]['rooms'] = [
        re.findall(r'\d\s[A-Za-z]+', feature.text)[0] for feature in bs_object \
            .find("div", {"data-testid": "property-features"}) \
            .findAll("span", {"data-testid": "property-features-text-container"})
    ]
    
    '''
    property_metadata[property_url]['desc'] = re \
        .sub(r'<br\/>', '\n', str(bs_object.find("p"))) \
        .strip('</p>')

# output to example json in data/raw/

with open('/Users/tianhao/ads_asmt2/furnished.json', 'w') as f:
    dump(property_metadata, f)

IndexError: list index out of range

# find feature = furnished

In [8]:
"""
A very simple and basic web scraping script. Feel free to
use this as a source of inspiration, but, make sure to attribute
it if you do so.
This is by no means production code.
"""
# built-in imports
import re
from json import dump

from collections import defaultdict

# user packages
from bs4 import BeautifulSoup, SoupStrainer
from urllib.request import urlopen
import requests

# constants
BASE_URL = "https://www.domain.com.au"
N_PAGES = range(1, 2) # update this to your liking

# begin code
url_links = []
property_metadata = defaultdict(dict)
headers = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}
# generate list of urls to visit
for page in N_PAGES:
    url = BASE_URL + f"/rent/melbourne-region-vic/?sort=price-desc&page={page}"
    bs_object = BeautifulSoup(requests.get(url,headers = headers).text, "html.parser")
    # find the unordered list (ul) elements which are the results, then
    # find all href (a) tags that are from the base_url website.
    if bs_object is not None:
        index_links = bs_object \
            .find(
                "ul",
                {"data-testid": "results"}
            ) \
            .findAll(
                "a",
                href=re.compile(f"{BASE_URL}/*") # the `*` denotes wildcard any
            )

    for link in index_links:
        # if its a property address, add it to the list
        if 'address' in link['class']:
            url_links.append(link['href'])



In [9]:
# for each url, scrape some basic metadata
for property_url in url_links[1:]:
    bs_object = BeautifulSoup(requests.get(property_url,headers = headers).text, "html.parser")

    # looks for the header class to get property name
    property_metadata[property_url]['name'] = bs_object \
        .find("h1", {"class": "css-164r41r"}) \
        .text

    # looks for the div containing a summary title for cost
    property_metadata[property_url]['cost_text'] = bs_object \
        .find("div", {"data-testid": "listing-details__summary-title"}) \
        .text

    # extract coordinates from the hyperlink provided
    # i'll let you figure out what this does :P
    property_metadata[property_url]['coordinates'] = [
        float(coord) for coord in re.findall(
            r'destination=([-\s,\d\.]+)', # use regex101.com here if you need to
            bs_object \
                .find(
                    "a",
                    {"target": "_blank", 'rel': "noopener noreferer"}
                ) \
                .attrs['href']
        )[0].split(',')
    ]
    '''
    property_metadata[property_url]['rooms'] = [
        re.findall(r'\d\s[A-Za-z]+', feature.text)[0] for feature in bs_object \
            .find("div", {"data-testid": "property-features"}) \
            .findAll("span", {"data-testid": "property-features-text-container"})
    ]
    
    '''
    property_metadata[property_url]['desc'] = re \
        .sub(r'<br\/>', '\n', str(bs_object.find("p"))) \
        .strip('</p>')

# output to example json in data/raw/

with open('/Users/tianhao/ads_asmt2/unfiltered.json', 'w') as f:
    dump(property_metadata, f)

In [13]:
import pandas as pd
df_furnished = pd.read_json('furnished.json')
df_raw = pd.read_json('unfiltered.json')

In [16]:
df_furnished = df_furnished.T
df_raw = df_raw.T

In [21]:
df_furnished = df_furnished.reset_index(drop=True)
df_raw = df_raw.reset_index(drop=True)

,name,cost_text,coordinates,desc
0,9 Lansdowne Street Blairgowrie VIC 3942,"$5,000 per week","[-38.372703, 144.7856897]",Phone enquiry code for this property : 2751
1,52 Black St Brighton VIC 3186,From $5500 per week,"[-37.9159452, 144.9989003]","class=""css-dxogle"">* Unverified feature<svg a..."
2,440 Keys Road Flinders VIC 3929,"$4,000.00","[-38.4788173, 144.9643428]",Boasting sublime architectural design and beau...
3,901 & 902/85 Market Street South Melbourne VIC...,$3750.00 per week,"[-37.8301164, 144.9569041]","class=""css-dxogle"">* Unverified feature<svg a..."
4,Brighton VIC 3186,Fine Furnished Residences at Westprecinct,"[-37.9043383, 144.9931825]",*Private inspections only at suggested times o...
5,7302/462 Elizabeth Street Melbourne VIC 3000,"$3,100 per week","[-37.807859, 144.960611]","class=""css-dxogle"">* Unverified feature<svg a..."
6,1 Bronte Court Hampton VIC 3188,"$3,000 per week","[-37.9399869, 145.0199068]",**FULLY FURNISHED**
7,Brighton VIC 3186,Fine Furnished Residences @ Westprecinct,"[-37.9044002, 144.9997396]",Welcome home to the GOLD STANDARD in furnished...
8,119-121 Palmerston Street Carlton VIC 3053,$2600 per week,"[-37.7965569, 144.9711271]",Please note that in accordance with the govern...
9,Blairgowrie VIC 3942,"$2,500 per week","[-38.3645949, 144.7686693]",Phone enquiry code for this property : 1942
